<h1 style="color:darkblue; font-size:32px;">פרויקט ניתוח נתונים מתקדם בפייתון</h1>


<h1 style="color:darkblue; font-size:15px;">קבוצה מספר: 16</h1>
<h1 style="color:darkblue; font-size:15px;">גל פלג  </h1>
<h1 style="color:darkblue; font-size:15px;">אוהב זך   </h1>
<h1 style="color:darkblue; font-size:15px;">אזורי שכונות: צפון יפו,נחלת יצחק,מרכז יפו מזרחית לשדרות ירושלים  </h1>

יבוא של הספריה שבה נשתמש

In [4]:
!pip install requests beautifulsoup4


בדיקה האם בקשת הגישה לאתר באמצעות דפדפן הכרום עובדת או לא והדפסה בהתאם

In [6]:
import requests
from bs4 import BeautifulSoup

url = "https://www.ad.co.il/nadlanrent?sp275=17413&sp277=17675"

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'} # This is chrome, you can set whatever browser you like

response = requests.get(url,headers= headers)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")


Success


בניית פונקציה הסורקת את עמוד התוצאות, מזהה מודעות נדל"ן אמיתיות , ושולפת את הקישורים הישירים של כל מודעה לרשימה מסודרת

In [8]:
def get_real_estate_links(url):
    import re
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
    }
    
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("❌ שגיאה בטעינת הדף")
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    elements = soup.find_all('div', class_=['card-block', 'hovered'])

    base_url = "https://www.ad.co.il"
    ad_links = []

    for element in elements:
        text = element.get_text()

        # סינון לפי סימנים של מודעת נדל"ן (שטח או חדרים)
        if "מ\"ר" not in text and "חד'" not in text:
            continue

        link_tag = element.find('a', href=True)
        if not link_tag:
            continue

        href = link_tag['href']
        if re.match(r"^/ad/\d+$", href):
            full_url = base_url + href
            ad_links.append(full_url)

    print(f"✅ נמצאו {len(ad_links)} מודעות נדל\"ן אמיתיות")
    return ad_links


 בניית פונקציה הקוראת את הפרמטרים הנדרשים מכל מודעה ומכניסה את הערכים למילון הפונקציה גם מסדרת קריאה של משתנים מסויימים לצורה תקינה ומשנה לשמות המשתנים הדרושים במטלה

In [10]:
def get_property_info(property_link, api_key=None):
    import requests
    from bs4 import BeautifulSoup
    from datetime import datetime
    import re
    import json

    def get_distance(location1, api_key):
        url = "https://routes.googleapis.com/directions/v2:computeRoutes"
        headers = {
            'Content-Type': 'application/json',
            'X-Goog-Api-Key': api_key,
            'X-Goog-FieldMask': 'routes.distanceMeters'
        }
        body = {
            "origin": {"address": location1},
            "destination": {"address": "כיכר דיזנגוף, תל אביב"},
            "travelMode": "DRIVE"
        }
        response = requests.post(url, headers=headers, json=body)
        if response.status_code == 200:
            try:
                data = response.json()
                return data['routes'][0]['distanceMeters']
            except Exception as e:
                print("שגיאה ב-JSON:", e)
                return None
        else:
            print("סטטוס שגוי:", response.status_code)
            return None

    property_dict = dict()

    try:
        response = requests.get(property_link)
        if response.status_code != 200:
            return property_dict

        soup = BeautifulSoup(response.content, 'html.parser')
        tables = soup.select('table.table-sm')

        for table in tables:
            rows = table.find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) == 2:
                    key = cols[0].get_text(strip=True)
                    value = cols[1].get_text(strip=True)

                    if key in ["אזור", "עיר"]:
                        continue

                    if key in ["מרפסות", "מרפסת שמש"]:
                        key = "מרפסת"

                    if key == "תאריך כניסה":
                        if "מייד" in value or "מידי" in value:
                            property_dict["מספר ימים לכניסה"] = 0
                        else:
                            try:
                                entry_date = datetime.strptime(value, "%d/%m/%Y")
                                days_until_entry = (entry_date - datetime.now()).days
                                property_dict["מספר ימים לכניסה"] = max(days_until_entry, 0)
                            except:
                                property_dict["מספר ימים לכניסה"] = None

                    elif key == "ארנונה בחודש":
                        property_dict["ארנונה חודשית"] = value

                    elif key == "חדרים":
                        property_dict["מספר חדרים"] = value

                    elif key == "קומה":
                        text = value
                        if "קרקע" in text:
                            property_dict["קומה"] = 0
                        else:
                            match = re.search(r"(\d+)", text)
                            property_dict["קומה"] = int(match.group(1)) if match else text

                        match_total = re.search(r"מתוך\s*(\d+)", text)
                        if match_total:
                            property_dict["קומות בבניין"] = int(match_total.group(1))

                    else:
                        property_dict[key] = value

        features_divs = soup.find_all('div', class_='card-icon')
        for feature_div in features_divs:
            feature_name_tag = feature_div.find('span')
            if not feature_name_tag:
                continue

            feature_name = feature_name_tag.get_text(strip=True)
            is_active = 'disabled' not in feature_div.get('class', [])

            if feature_name == "מרוהטת":
                property_dict["מרוהטת"] = int(is_active)
            elif feature_name == "מזגן":
                property_dict["מיזוג"] = int(is_active)
            elif feature_name == "חניה":
                property_dict["חניה"] = int(is_active)
            elif feature_name == "ממ\"ד":
                property_dict["ממ\"ד"] = int(is_active)
            elif feature_name in ["מרפסת", "מרפסת שמש", "מרפסות"]:
                if "מרפסת" not in property_dict:
                    property_dict["מרפסת"] = 0
                if is_active:
                    property_dict["מרפסת"] = 1
            elif feature_name == "נגישות":
                property_dict["נגישות לנכים"] = int(is_active)
            elif feature_name == "סורגים":
                property_dict["סורגים"] = int(is_active)
            elif feature_name == "מעלית":
                property_dict["מעלית"] = int(is_active)
            elif feature_name == "מחסן":
                property_dict["מחסן"] = int(is_active)
            elif feature_name == "משופצת":
                property_dict["משופצת"] = int(is_active)

        price_tags = soup.select("h2.card-title")
        if len(price_tags) > 1:
            price_text = price_tags[1].get_text(strip=True).replace("₪", "").replace(",", "").strip()
            try:
                property_dict["מחיר"] = int(price_text)
            except:
                property_dict["מחיר"] = price_text

        description_tag = soup.find("p", class_="text-word-break")
        if description_tag:
            property_dict["תיאור"] = description_tag.get_text(strip=True)

        image_tags = soup.select("img.desktop-thumbnail")
        property_dict["מספר תמונות"] = len(image_tags)

        if api_key and "כתובת" in property_dict:
            full_address = property_dict["כתובת"] + ", תל אביב"
            distance = get_distance(full_address, api_key)
            if distance:
                property_dict["מרחק מכיכר דיזנגוף"] = round(distance / 1000, 2)

        # הוספת ערכים ברירת מחדל לשדות חסרים
        if "שטח גינה" not in property_dict:
            property_dict["שטח גינה"] = 0
        if "ארנונה חודשית" not in property_dict:
            property_dict["ארנונה חודשית"] = None
        if "מספר ימים לכניסה" not in property_dict:
            property_dict["מספר ימים לכניסה"] = None


        # מיפוי לשמות אנגליים
        field_map = {
            'פרטי הנכס': 'property_type',
            'שכונה': 'neighborhood',
            'כתובת': 'address',
            'מספר חדרים': 'room_num',
            'קומה': 'floor',
            'שטח בנוי': 'area',
            'שטח גינה': 'garden_area',
            'מספר ימים לכניסה': 'days_to_enter',
            'תשלומים בשנה': 'num_of_payments',
            'ארנונה חודשית': 'monthly_arnona',
            'ועד בית בחודש': 'building_tax',
            'קומות בבניין': 'total_floors',
            'תיאור': 'description',
            'חניה': 'has_parking',
            'מחסן': 'has_stotsge',
            'מעלית': 'elevator',
            'מיזוג': 'ac',
            'נגישות לנכים': 'handicap',
            'סורגים': 'has_bars',
            'ממ"ד': 'has_safe_room',
            'מרפסת': 'has_balcon',
            'מרוהטת': 'is_furnished',
            'משופצת': 'is_renovated',
            'מחיר': 'price',
            'מספר תמונות': 'num_of_images',
            'מרחק מכיכר דיזנגוף': 'distance_from_center'
        }

        reordered_dict = {eng_key: property_dict.get(heb_key, None) for heb_key, eng_key in field_map.items()}
        return reordered_dict

    except Exception as e:
        print(f"שגיאה: {e}")
        return property_dict

def get_all_ads_from_page(url, api_key=None):
    from time import sleep

    ads_data = []

    # שימוש בפונקציה שמבצעת סינון של פרסומות
    links = get_real_estate_links(url)

    for i, link in enumerate(links, 1):
        try:
            print(f"({i}/{len(links)}) שואב פרטים מ: {link}")
            data = get_property_info(link, api_key=api_key)
            if data:
                ads_data.append(data)
                sleep(0.5)  # מנוחה בין בקשות
        except Exception as e:
            print(f"שגיאה בקישור {link}: {e}")
            continue

    return ads_data

פונקציה העוברת על כל עמוד של אזור ובתוכה מפעילה את הפונקציה שוראת כל מודעה בעמוד וכך אנו צוברים את הנתונים על כל  מודעה למשתנה

In [12]:
def get_all_ads_from_page(url, api_key=None):
    from time import sleep

    ads_data = []

    # שימוש בפונקציה שמבצעת סינון של פרסומות
    links = get_real_estate_links(url)

    for i, link in enumerate(links, 1):
        try:
            print(f"({i}/{len(links)}) שואב פרטים מ: {link}")
            data = get_property_info(link, api_key=api_key)
            if data:
                ads_data.append(data)
                sleep(0.5)  # מנוחה בין בקשות
        except Exception as e:
            print(f"שגיאה בקישור {link}: {e}")
            continue

    return ads_data





ביצוע הקריאה של הפונקציה בשילוב המפתח  על כל קישור לאזור שיש לנו

In [14]:
api_key = "enter api"

url1 = "https://www.ad.co.il/nadlanrent?sp275=17413&sp276=17414&sp277=17675"
url2 = "https://www.ad.co.il/nadlanrent?sp275=17413&sp277=17913"
url3 = "https://www.ad.co.il/nadlanrent?sp275=17413&sp277=17821"

ads_data_1 = get_all_ads_from_page(url1, api_key)
ads_data_2 = get_all_ads_from_page(url2, api_key)
ads_data_3 = get_all_ads_from_page(url3, api_key)

# מאחדים את כל הנתונים לרשימה אחת
ads_data = ads_data_1 + ads_data_2 + ads_data_3


✅ נמצאו 21 מודעות נדל"ן אמיתיות
(1/21) שואב פרטים מ: https://www.ad.co.il/ad/16200424
(2/21) שואב פרטים מ: https://www.ad.co.il/ad/16200425
(3/21) שואב פרטים מ: https://www.ad.co.il/ad/16191364
(4/21) שואב פרטים מ: https://www.ad.co.il/ad/14966680
(5/21) שואב פרטים מ: https://www.ad.co.il/ad/16191322
(6/21) שואב פרטים מ: https://www.ad.co.il/ad/16191339
(7/21) שואב פרטים מ: https://www.ad.co.il/ad/16188384
(8/21) שואב פרטים מ: https://www.ad.co.il/ad/16188382
(9/21) שואב פרטים מ: https://www.ad.co.il/ad/15888752
(10/21) שואב פרטים מ: https://www.ad.co.il/ad/15873579
(11/21) שואב פרטים מ: https://www.ad.co.il/ad/15848194
(12/21) שואב פרטים מ: https://www.ad.co.il/ad/15120590
(13/21) שואב פרטים מ: https://www.ad.co.il/ad/14883933
(14/21) שואב פרטים מ: https://www.ad.co.il/ad/14344961
(15/21) שואב פרטים מ: https://www.ad.co.il/ad/13552188
(16/21) שואב פרטים מ: https://www.ad.co.il/ad/13407283
(17/21) שואב פרטים מ: https://www.ad.co.il/ad/13195268
(18/21) שואב פרטים מ: https://www.ad.co.il

שמירה בדאטה פריים את כל הנתונים

In [16]:
import pandas as pd
df=pd.DataFrame(ads_data)

שינוי לטייפ המתאים של כל עמודה בדאטה פריים

In [18]:

# המרת טיפוסים לפי הטבלה מהמטלה
str_cols = ['property_type', 'neighborhood', 'address', 'description']
float_cols = ['room_num', 'price', 'distance_from_center']
int_cols = [
    'floor', 'area', 'garden_area', 'days_to_enter', 'num_of_payments',
    'monthly_arnona', 'building_tax', 'total_floors', 'num_of_images'
]
binary_cols = [
    'has_parking', 'has_stotsge', 'elevator', 'ac', 'handicap',
    'has_bars', 'has_safe_room', 'has_balcon',
    'is_furnished', 'is_renovated'
]

# המרה לעמודות מסוג string
for col in str_cols:
    df[col] = df[col].astype('string')

# המרה לעמודות מסוג float
for col in float_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype(float)

# המרה לעמודות מסוג Int64
for col in int_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

# המרה לעמודות בינאריות (0 או 1)
for col in binary_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').astype(int)

# הדפסת טיפוסים לבדיקה
print(df.dtypes)



property_type           string[python]
neighborhood            string[python]
address                 string[python]
room_num                       float64
floor                            Int64
area                             Int64
garden_area                      Int64
days_to_enter                    Int64
num_of_payments                  Int64
monthly_arnona                   Int64
building_tax                     Int64
total_floors                     Int64
description             string[python]
has_parking                      int32
has_stotsge                      int32
elevator                         int32
ac                               int32
handicap                         int32
has_bars                         int32
has_safe_room                    int32
has_balcon                       int32
is_furnished                     int32
is_renovated                     int32
price                          float64
num_of_images                    Int64
distance_from_center     

שמירה לקובץ CSV

In [20]:


# שמירה לקובץ
df.to_csv("1מודעות_השכרה.csv", index=False, encoding='utf-8-sig')
print("הקובץ נשמר בהצלחה!")


הקובץ נשמר בהצלחה!


In [21]:
df  # מדפיס את כל הטבלה (רק אם היא לא עצומה)


,property_type,neighborhood,address,room_num,floor,area,garden_area,days_to_enter,num_of_payments,monthly_arnona,...,ac,handicap,has_bars,has_safe_room,has_balcon,is_furnished,is_renovated,price,num_of_images,distance_from_center
0,דירה,צפון יפו,שמעון הצדיק 7,3.0,0,60,30,0,12,200,...,1,0,0,0,1,1,1,7200.0,15,4.59
1,דירה,צפון יפו,הדואר 14,4.0,0,90,0,0,12,200,...,1,0,0,0,1,1,1,8200.0,9,4.57
2,דירה,צפון יפו,שמעון בן שטח 14,3.0,2,105,20,0,12,350,...,1,1,0,1,1,1,1,13000.0,5,4.35
3,דירה,צפון יפו,אברבנאל 4,1.0,0,35,0,0,12,300,...,1,0,1,0,0,0,1,4500.0,2,5.17
4,דירה,צפון יפו,בר הופמן 6,3.0,6,66,0,<NA>,12,<NA>,...,1,1,0,1,1,1,1,8700.0,6,4.68
5,דירה,צפון יפו,<NA>,3.5,5,122,0,0,12,<NA>,...,1,1,0,1,1,0,1,10700.0,4,NaN
6,סטודיו/לופט,צפון יפו,מעון 10,1.0,0,40,20,0,6,<NA>,...,0,0,0,0,1,0,0,4400.0,1,4.79
7,סטודיו/לופט,צפון יפו,מעון 10,1.0,0,40,20,0,6,<NA>,...,0,0,0,0,1,0,0,4400.0,0,4.79
8,דירה,צפון יפו,קומפרט 7,3.0,10,80,0,0,12,100,...,1,0,0,1,1,0,1,10700.0,2,4.86
9,דירה,צפון יפו,אברבנאל 4,3.0,0,60,0,0,12,300,...,1,0,1,0,0,0,1,6500.0,5,5.17
